# 1-Introduction

## Definitions:

For the sake of consistency, it is important to lay out some terms that will be referred to consistently in this project.

1. "Course": Defined by the track, race type and distance.

2. "Race": Defined by the track, date and program number.  It will include the data, "track", horse-trainer pairing and outcome.

3. "Path": Defined by the horses within each race.  If available, it will include the long/lat-coordinates which are used to generate statistics.

# 2-Library, Functions & Data Upload

## Libraries

In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from lxml import etree, objectify

https://www.google.com/search?client=safari&rls=en&q=xml+python&ie=UTF-8&oe=UTF-8

In [2]:
import geopandas
import shapely
from shapely.geometry import Point, MultiLineString, LineString
from haversine import haversine, haversine_vector

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
#from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset
from sklearn.metrics import silhouette_score

## User-Defined Functions

In [ ]:
def analyze_path(points):
    points_sorted = points.sort_values(['trakus_index'])
    max_time = points['trakus_index'].max()
    dists = []
    deltas_lon = []
    deltas_lat = []
    vls = []
    accls = []
    dirs = []
    dirs_delta = []
    lines = []
    for i in range(0,len(points)-1):
        t = points_sorted.iloc[i+1]['trakus_index']
        p1 = points_sorted.iloc[i].Point
        p2 = points_sorted.iloc[i+1].Point
        cord1 = p1.coords[0]
        cord2 = p2.coords[0]
        lines.append(LineString([p1,p2]))
        dist = haversine(cord2,cord1)
        delta_lon = cord2[0] - cord1[0]
        delta_lat = cord2[1] - cord1[1]
        deltas_lon.append(delta_lon)
        deltas_lat.append(delta_lat)
        dists.append(dist)
        vls.append(dist/0.25)
        v_delta = vls[-1] - vls[-2] if i > 1 else vls[-1]
        accl = v_delta/0.25
        accls.append(accl)
        direction = np.arctan2(delta_lon,delta_lat)
        dir_delta = direction - dirs[-1] if i > 0 else 0
        dirs.append(direction)
        dirs_delta.append(dir_delta)


    results = {'distances':dists,
               'speed':vls,'acceleration':accls,'direction':dirs,
               'delta_lat':deltas_lat,'delta_lon':deltas_lon,'delta_direction':dirs_delta
               }
    path = MultiLineString(lines)    
    return path, results
    

In [ ]:
def calculate_metrics(run):
    pass

In [ ]:
def create_backdata(id,date,data,features=None):
    pass

In [ ]:
def create_sequences(df, seq_length):
    xs = []
    ys = []
    for trajectory_id in df['trajectory_id'].unique():
        df_trajectory = df[df['trajectory_id'] == trajectory_id]
        for i in range(len(df_trajectory) - seq_length):
            x = df_trajectory[i:(i + seq_length)][['longitude', 'latitude', 'speed', 'acceleration', 'delta_direction', 'distance_to_leader']].values
            y = df_trajectory.iloc[i + seq_length][['longitude', 'latitude']].values
            xs.append(x)
            ys.append(y)
    return np.array(xs), np.array(ys)

def create_sequences_and_targets(data, seq_length):
    """Creates sequences of data and corresponding next-step targets."""
    xs = []
    ys = []
    # Group data by race and program number to process sequences within each horse's track
    grouped = data.groupby(['race_number', 'program_number'])
    for name, group in grouped:
        # Extract features (longitude and latitude)
        features = group[['normalized_longitude', 'normalized_latitude']].values
        for i in range(len(features) - seq_length):
            # Create input sequence
            x = features[i:(i + seq_length)]
            # Create target (next step)
            y = features[i + seq_length]
            xs.append(x)
            ys.append(y)

## Datasets

In [9]:
files = os.listdir('Data/2023 Result Charts')
print(len(files))

4906


In [50]:
base = objectify.parse('Data/2023 Result Charts/'+files[0])

TypeError: Argument 'element' has incorrect type (expected lxml.etree._Element, got str)

In [56]:
base.getroot().getchildren()[0]

<Element TRACK at 0x11096d700>

In [61]:
tree = etree.parse('Data/2023 Result Charts/'+files[0])
root = tree.getroot()

In [63]:
tree.findall('.//RACE')

[<Element RACE at 0x2879eaec0>,
 <Element RACE at 0x2879eba40>,
 <Element RACE at 0x110a00f00>,
 <Element RACE at 0x110a00d80>,
 <Element RACE at 0x110a013c0>,
 <Element RACE at 0x110a01ac0>,
 <Element RACE at 0x110a01000>]

In [ ]:
root.getchildren()[0].tag

'TRACK'

In [1]:
heads = ['track','date',
         'race_number','program_number',
         'trakus_index','latitude','longitude',
         'race_distance','course_type','track_condition',
         'run_up_distance','race_type','post_time',
         'purse','weight_carried','jockey',
         'odds','finish'
         ]

In [ ]:
data = pd.read_csv('Data/nyra_2019_complete.csv',header=None)
data.columns = heads
data

/var/folders/7r/r2s5qqq505q1dy3w3pxj6rnc0000gn/T/ipykernel_30084/639767100.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Data/nyra_2019_complete.csv',header=None)


ValueError: Length mismatch: Expected axis has 18 elements, new values have 17 elements

In [ ]:
starts = pd.read_csv('data/nyra_start_table_new.csv')

# 3-Basic EDA

## Tracks

## Horses & Trainers

In [ ]:
jockeys = starts['jockey'].value_counts()
jockeys.plot.hist()
jockeys

In [ ]:
horses = starts['horse_id'].value_counts()
horses.plot.hist()
horses

In [ ]:
pairings = starts[['horse_id','jockey']].value_counts()
pairings.plot.hist()
pairings

## Races

## Runs

## Bad Data

# 4-Feature Engineering/Wrangling

# 5-Full EDA

# 6-Modeling & Analysis

## Basic Regression

## Clustering

Which features should be used?

In [ ]:
features = ['distance', 'speed', 'acceleration']

Prepare for clustering by grouping runs; converting into type for TS analyses; handling bad data.

In [ ]:
grouped_data = df.groupby(['track_id', 'race_date', 'race_number', 'program_number'])[features].apply(lambda x: x.values)
grouped_data = grouped_data.tolist()
X = to_time_series_dataset(grouped_data)
X[np.isnan(X)] = 0
X[np.isinf(X)] = 0

Define the range for k (# of clusters) we will be examining.

In [ ]:
silhouette_scores = []
range_n_clusters = [2,4,5,6,8,10,15,20]

Check to make sure silhouette score can be calculated for clustering--i.e. there are at least 2 clusters and more than 1 sample in each cluster.

In [ ]:
for n_clusters in range_n_clusters:
    #print(f"Trying {n_clusters} clusters...")
    km_dtw = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=10, random_state=42)
    labels = km_dtw.fit_predict(X)
    # 
    # Check if there is more than one cluster and each cluster has more than one sample
    if len(np.unique(labels)) > 1 and min(np.bincount(labels)) > 1:
        score = silhouette_score(X.reshape(X.shape[0], -1), labels) # Reshape for silhouette_score
        silhouette_scores.append(score)
        #print(f"Silhouette score for {n_clusters} clusters: {score}")
    else:
        silhouette_scores.append(-1) # Append a low score if conditions are not met
        #print(f"Could not compute silhouette score for {n_clusters} clusters.")

Identify the optimal # of clusters amongst the range explored and fit a new optimal model.

In [ ]:
optimal_n_clusters = range_n_clusters[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_n_clusters}")

In [ ]:
km_dtw_optimal = TimeSeriesKMeans(n_clusters=optimal_n_clusters, metric="dtw", max_iter=10, random_state=42)
cluster_labels = km_dtw_optimal.fit_predict(X)

Analyze the results:

In [ ]:
group_keys = df.groupby(['track_id', 'race_date', 'race_number', 'program_number']).groups.keys()
group_key_list = list(group_keys)

# Create a list of (group_key, label) pairs
group_labels = list(zip(group_key_list, cluster_labels))

# Create a dictionary mapping group key to label
label_dict = {key: label for key, label in group_labels}

In [ ]:
df['group_key'] = list(zip(df['track_id'], df['race_date'], df['race_number'], df['program_number']))

# Add the cluster label to the original DataFrame
df['cluster_label'] = df['group_key'].map(label_dict)

# Drop the temporary group_key column
df = df.drop(columns=['group_key'])

display(df.head())

## Geospatial Deep Learning

In [ ]:
seq_length = 20
X_train, y_train = create_sequences_and_targets(train_df, seq_length)
X_val, y_val = create_sequences_and_targets(val_df, seq_length)

In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)

In [ ]:
inputs = Input(shape=(seq_length, 2))
lstm_out = LSTM(64, return_sequences=True)(inputs)

attention_output = Attention()([lstm_out, lstm_out])
merged_output = keras.layers.concatenate([lstm_out, attention_output])

In [ ]:
pooled_attention = keras.layers.GlobalAveragePooling1D()(attention_output)
outputs = Dense(2)(pooled_attention)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# 7-Conclusions

## Research & Resources